TASK1: INTRODUCTION

In [ ]:
!pip install nn_utils

     |████████████████████████████████| 327kB 2.7MB/s 


In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed, Dense, Dropout, SimpleRNN, RepeatVector
from tensorflow.keras.callbacks import EarlyStopping, LambdaCallback
import nn_utils
from termcolor import colored

TASK2: GENERATE DATA

In [ ]:
all_chars='0123456789+'

In [ ]:
num_feature=len(all_chars)
print('number of features:',num_feature)
char_to_index=dict((c,i) for i,c in enumerate(all_chars))
index_to_char=dict((i,c) for i,c in enumerate (all_chars))

number of features: 11


In [ ]:
def generate_data():
  first=np.random.randint(0,100)
  second=np.random.randint(0,100)
  example=str(first)+ '+'+str(second)
  label=str(first+second)
  return example,label
generate_data()  

('50+31', '81')

TASK3: CREATE THE MODEL

consider two reviews:

review 1: This movie is not terrible at all.

review 2: This movie is pretty descent.

In [ ]:
hidden_units=128
max_time_steps=5
model=Sequential([
                  SimpleRNN(hidden_units, input_shape=(None, num_feature)),
                  RepeatVector(max_time_steps),
                  SimpleRNN(hidden_units, return_sequences=True),
                  TimeDistributed(Dense(num_feature, activation='softmax'))
])
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 128)               17920     
_________________________________________________________________
repeat_vector (RepeatVector) (None, 5, 128)            0         
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 5, 128)            32896     
_________________________________________________________________
time_distributed (TimeDistri (None, 5, 11)             1419      
Total params: 52,235
Trainable params: 52,235
Non-trainable params: 0
_________________________________________________________________


TASK4: VECTORISE AND DEVECTORISE DATA

In [ ]:
def vectorize_example(example, label):
  x=np.zeros((max_time_steps, num_feature))
  y=np.zeros((max_time_steps, num_feature))

  diff_x=max_time_steps - len(example)
  diff_y=max_time_steps - len(label)

  for i,c in enumerate(example):
    x[i+diff_x, char_to_index[c]]=1
  for i in range(diff_x):
    x[i,char_to_index['0']]=1  
  for i,c in enumerate(label):
    y[i+diff_y, char_to_index[c]]=1
  for i in range(diff_y):
    y[i,char_to_index['0']]=1 
  return x, y
m=1  
e,m=generate_data()
print(e, m)

x, y = vectorize_example(e, m)
print(x.shape, y.shape)


74+82 156
(5, 11) (5, 11)


In [ ]:
def devectorize_example(example):
  result=[index_to_char[np.argmax(vec)] for i,vec in enumerate(example)]
  return''.join(result)

devectorize_example(x)  

'74+82'

In [ ]:
  devectorize_example(y)

'00156'

TASK5:CREATE DATASET

In [ ]:
def create_dataset(num_examples=2000):
  x=np.zeros((num_examples, max_time_steps, num_feature))
  y=np.zeros((num_examples, max_time_steps, num_feature))
  for i in range(num_examples):
    m=1
    e, m = generate_data()
    e_v, m_v = vectorize_example(e,m)
    x[i]=e_v
    y[i]=m_v
  return x, y
x,y= create_dataset()
print(x.shape,y.shape)    

(2000, 5, 11) (2000, 5, 11)


In [ ]:
devectorize_example(x[0])

'54+57'

In [ ]:
devectorize_example(y[0])

'00111'

TASK6:TRAINING THE MODEL

In [ ]:

m_cb = LambdaCallback(
    on_epoch_end=lambda
    e, m: print('{:.2f}'.format(m['val_accuracy']), end='-')
)
es_cb=EarlyStopping(monitor='val_loss',patience=10)
model.fit(x, y, epochs=500, batch_size=256, validation_split=0.2,
          verbose=False, callbacks=[es_cb,m_cb])

0.55-0.60-0.62-0.61-0.62-0.63-0.63-0.63-0.64-0.63-0.64-0.65-0.66-0.65-0.65-0.66-0.66-0.67-0.67-0.68-0.69-0.70-0.70-0.70-0.71-0.72-0.72-0.72-0.71-0.72-0.74-0.74-0.75-0.74-0.75-0.75-0.74-0.75-0.77-0.76-0.77-0.76-0.77-0.77-0.78-0.78-0.79-0.80-0.80-0.80-0.80-0.81-0.82-0.82-0.82-0.83-0.83-0.84-0.84-0.85-0.85-0.86-0.87-0.86-0.87-0.88-0.87-0.88-0.88-0.87-0.88-0.89-0.89-0.89-0.90-0.90-0.90-0.90-0.91-0.90-0.91-0.91-0.91-0.91-0.92-0.92-0.92-0.92-0.93-0.92-0.92-0.93-0.92-0.92-0.93-0.93-0.93-0.93-0.93-0.93-0.94-0.93-0.93-0.94-0.94-0.93-0.94-0.94-0.94-0.94-0.94-0.94-0.94-0.94-0.94-0.94-0.94-0.94-0.94-0.94-0.94-0.94-0.94-0.94-0.94-0.94-0.94-0.94-0.94-0.94-0.94-0.94-0.94-0.94-0.94-0.94-0.94-0.94-0.94-0.95-0.94-0.94-0.94-0.94-0.95-0.94-0.95-0.94-0.95-0.94-0.94-0.95-0.95-0.95-0.95-0.94-0.95-0.95-0.95-

In [ ]:
x_test, y_test= create_dataset(10)
preds = model.predict(x_test)

for i,pred in enumerate(preds):
  y=devectorize_example(y_test[i])
  y_hat=devectorize_example(pred)
  col='green'
  if y != y_hat:
    col='red'
  out='Input:'+devectorize_example(x_test[i])+'out:'+y+'pred:'+y_hat 
  print(colored(out,col))

Input:04+26out:00030pred:00030
Input:62+15out:00077pred:00077
Input:10+95out:00105pred:00106
Input:90+62out:00152pred:00152
Input:49+69out:00118pred:00118
Input:22+50out:00072pred:00072
Input:54+61out:00115pred:00115
Input:49+56out:00105pred:00105
Input:47+33out:00080pred:00080
Input:49+36out:00085pred:00075
